In [2]:
import os
import logging

import psycopg2
import psycopg2.extensions
import pandas as pd
import numpy as np


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
logger.info("Создаём подключёние к Postgres")
params = {
    "host": '0.0.0.0',
    "port": '5432',
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

2018-08-25 23:01:19,495 : INFO : Создаём подключёние к Postgres


In [4]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()


In [8]:
cursor.execute("select * from product limit 10;")
[i for i in cursor.fetchall()]


[(1, 'Converge', 'Jane Doe', datetime.date(2001, 9, 4)),
 (2, 'Converge', 'You Fail Me', datetime.date(2004, 9, 20)),
 (3, 'Jack the Stripper', 'Raw Nerve', datetime.date(2013, 9, 23)),
 (4, 'Harlots', 'Betrayer', datetime.date(2007, 1, 1)),
 (5, 'Architects', 'Hollow Crown', datetime.date(2009, 1, 26)),
 (6, 'Protest the Hero', 'Kezia', datetime.date(2006, 6, 10)),
 (7, 'Loftus', 'Hugs & Drugs', datetime.date(2005, 1, 11)),
 (8,
  'Postmortem Promises',
  'Postmortem Promises [EP]',
  datetime.date(2007, 1, 1)),
 (9, 'From a Second Story Window', 'Delenda', datetime.date(2006, 7, 11)),
 (10, 'Eternal Lord', 'Blessed Be This Nightmare', datetime.date(2008, 3, 18))]

In [10]:
products = pd.read_sql('SELECT * FROM product', conn)
logger.info("Таблица продуктов")
print(products.head())


2018-08-26 00:18:16,412 : INFO : Таблица продуктов


   id             artist         album release_date
0   1           Converge      Jane Doe   2001-09-04
1   2           Converge   You Fail Me   2004-09-20
2   3  Jack the Stripper     Raw Nerve   2013-09-23
3   4            Harlots      Betrayer   2007-01-01
4   5         Architects  Hollow Crown   2009-01-26


In [11]:
sales = pd.read_sql('SELECT * FROM sales', conn)
logger.info("Таблица продаж")
print(sales.head())

2018-08-26 00:23:11,464 : INFO : Таблица продаж


   id  product_id  customer_id currency  price  quantity
0   1          14           32        $   9.43         1
1   2          13           57        $  29.25         1
2   3           5           33        $   2.34         1
3   4          11           69        $  20.12         1
4   5           7           67        $  28.03         1


In [33]:
customers = pd.read_sql('SELECT * FROM customers', conn)
logger.info("Таблица покупателей")
print(customers.head())

2018-08-26 01:03:48,608 : INFO : Таблица покупателей


   id first_name  last_name     bithday
0   1        Ann    Mccarty  1977-11-28
1   2    William     Spears  1987-06-14
2   3     Hector  Gillespie  1982-11-08
3   4     Cullen   Reynolds  1984-06-22
4   5      Nigel    Osborne  1980-09-19


In [34]:
addresses = pd.read_sql('SELECT * FROM addresses', conn)
logger.info("Таблица адресов")
print(addresses.head())

2018-08-26 01:04:12,023 : INFO : Таблица адресов


   id  customer_id                      country             city  \
0   1           47                  Philippines             Lang   
1   2           47                    Nicaragua  Rivi�re-du-Loup   
2   3           25  French Southern Territories           Rishra   
3   4           56                       Greece          Dhanbad   
4   5           22                       Latvia            Minto   

                                street  home            phone  
0            Ap #686-1230 Ipsum Avenue    10  (03590) 0693525  
1  P.O. Box 985, 6453 Penatibus Avenue     5   (057) 06960747  
2                    5385 Sociosqu St.    18  (0456) 06042383  
3                       618 Magnis St.    37   (068) 48393440  
4                   Ap #854-8810 A Ave    21  (00951) 5027843  


In [14]:
min_max_prods = sales.groupby(['product_id']).agg({'price': ['min', 'max']}).sort_values(by='product_id', ascending=False).head(5)
print(min_max_prods.head())

            price       
              min    max
product_id              
17          12.85  43.44
16          21.36  38.28
15          10.59  45.46
14           2.38  47.17
13          11.64  29.25


In [31]:
sum_customer_buys = sales.groupby(['customer_id']).agg({'price' : ['sum']}).sort_values(by='customer_id', ascending=False).head(20)
print(sum_customer_buys.head(5))

              price
                sum
customer_id        
70           137.53
69            69.43
68           104.88
67            29.58
66            43.58


In [44]:
query1 = customers.merge(addresses[addresses.country == 'Latvia'][['customer_id','id']], how='inner', left_on='id', right_on='customer_id')[['first_name', 'last_name']]
print(query1.head())


  first_name last_name
0     Juliet  Castillo


In [38]:
query2 = customers.merge(addresses, how='inner', left_on='id', right_on='customer_id').head()
print(query2.head())

   id_x first_name  last_name     bithday  id_y  customer_id  \
0     1        Ann    Mccarty  1977-11-28     6            1   
1     2    William     Spears  1987-06-14    72            2   
2     2    William     Spears  1987-06-14    80            2   
3     3     Hector  Gillespie  1982-11-08    22            3   
4     4     Cullen   Reynolds  1984-06-22     9            4   

             country          city                         street  home  \
0            Lesotho       Venezia                2752 Quam, Road    53   
1           Mongolia    Hunstanton            352-4950 Felis Road    39   
2  Equatorial Guinea        Arviat        Ap #855-9585 Cursus Rd.     3   
3          Lithuania     Candidoni  P.O. Box 956, 5185 Dolor, Av.    28   
4           Ethiopia  Acquafondata          484-7232 Accumsan Rd.    13   

             phone  
0  (0303) 31496842  
1  (07315) 8217288  
2  (0335) 86636122  
3  (0031) 39980692  
4  (030599) 382623  


In [46]:
query3 = sales[sales['product_id'].between(7, 12, inclusive=True)]
print(query3.head())

    id  product_id  customer_id currency  price  quantity
3    4          11           69        $  20.12         1
4    5           7           67        $  28.03         1
5    6           8           48        $  48.56         1
13  14           9           57        $  13.08         1
21  22           8           55        $  13.08         1
